## Import Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re
import string
import os
import random
import praw
import emoji
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore")

from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from collections import Counter, defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation as LDA
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nltk.download('punkt',)
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to C:\Users\Luis
[nltk_data]     Perez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Luis Perez\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Luis
[nltk_data]     Perez\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Luis Perez\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Luis
[nltk_data]     Perez\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [20]:
df = pd.read_csv('pre_processed.csv')



In [21]:
# Drop rows with NaN values in categories column
df = df.dropna(subset=['categories'])

# Keep only rows where categories contain 'restaurants' or 'food'
df = df[df['categories'].str.contains('restaurants|food', case=False, regex=True)]

# Get the length of the resulting dataframe
length = len(df)

print(f"Length of dataset: {length}")

Length of dataset: 239065


In [22]:
df = df.rename(columns={'text_processed': 'review', 'name': 'location'})


In [7]:
df['review'] = df['review'].astype(str)

In [8]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [9]:
sid = SentimentIntensityAnalyzer()

In [10]:
import re
import nltk

def split_into_phrases(text):
    sentences = nltk.sent_tokenize(text)
    phrases = []
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        # split sentence into phrases based on conjunctions, punctuation, and other rules
        conjunctions = ["for", "and", "nor", "but", "or", "yet", "so", "although", "though", "even though", "while", "if", "unless", "until", "since", "because", "as", "in order that", "so that", "that", "whether", "while", "who", "whom", "whose", "which", "what", "where", "when", "why", "how", "therefore", "furthermore", "however", "nevertheless", "moreover", "in addition", "on the other hand", "in contrast", "meanwhile", "conversely", "likewise", "nonetheless", "regardless", "thus", "to conclude", "to summarize", "in conclusion", "in summary", "to sum up", "to put it differently", "that is to say", "in other words", "for example", "such as", "including", "namely", "particularly", "notably", "chiefly", "mostly", "mainly", "primarily", "not only", "but also", "besides", "even", "indeed", "in fact", "to be sure", "above all", "certainly", "in any case", "in essence", "in particular", "in short", "in brief", "to this end", "with this in mind", "to begin with", "firstly", "secondly", "thirdly", "lastly", "finally", "meanwhile", "accordingly", "consequently", "hence", "otherwise", "thus", "thereby", "therefore", "whereas", "all in all", "altogether", "as a result", "at any rate", "in any event", "in brief", "in conclusion", "on balance", "on the whole", "to sum up", "to summarize", "after all", "as has been noted", "in a word", "in brief", "in conclusion", "in essence", "in summary", "on balance", "to conclude", "to sum up", "to summarize"]
        phrase = ""
        for i, token in enumerate(tokens):
            if token in conjunctions or token in ".,;:":
                phrases.append(phrase.strip())
                phrase = ""
            elif i > 0 and re.match(r"^[A-Z]{1}[a-z]+$", token) and re.match(r"^[a-z]+$", tokens[i-1]):
                phrases.append(phrase.strip())
                phrase = token
            elif i > 0 and token == "'" and tokens[i-1].endswith("s"):
                phrase += tokens[i-1]
            else:
                phrase += " " + token
        if phrase:
            phrases.append(phrase.strip())
    return phrases


In [11]:
df['phrases'] = df['review'].apply(lambda x: split_into_phrases(x))
df

,business_id,location,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,useful,funny,cool,text,date,review,tokens,num_tokens,has_emoji,phrases
31,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,0,0,0,What a great addition to the Funk Zone! Grab ...,2016-10-13 22:50:47,what a great addition to the funk zone grab a ...,"['great', 'addition', 'funk', 'zone', 'grab', ...",24,False,"[, a great addition to the funk zone grab a bi..."
32,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,0,0,0,Possibly the best breakfast sandwich EVER. On...,2017-01-15 00:36:04,possibly the good breakfast sandwich ever on a...,"['possibly', 'best', 'breakfast', 'sandwich', ...",9,False,[possibly the good breakfast sandwich ever on ...
33,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,0,0,0,Pricey ( a ham and cheese croissant was $5.50)...,2018-07-17 00:20:41,pricey a ham and cheese croissant be as with a...,"['pricey', 'ham', 'cheese', 'croissant', '550'...",46,False,"[pricey a ham, cheese croissant be, with anywh..."
34,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,0,1,1,Are you kidding me? This is without a doubt so...,2018-01-09 20:44:47,be you kid I this be without a doubt some of t...,"['kidding', 'without', 'doubt', 'best', 'food'...",30,False,[be you kid I this be without a doubt some of ...
35,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,3,0,0,"Super slow service, ""salads"" are not very good...",2017-02-09 19:09:31,super slow service be not very good and really...,"['super', 'slow', 'service', 'salad', 'good', ...",9,False,"[super slow service be not very good, really o..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348844,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,0,0,0,This is one of my new favorite pizza joints he...,2015-04-11 04:43:19,this be one of my new favorite pizza joint her...,"['one', 'new', 'favorite', 'pizza', 'joint', '...",69,False,[this be one of my new favorite pizza joint he...
348845,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,0,0,0,Ordered vegan chicken and Daiya cheese and onl...,2015-05-26 15:27:36,order vegan chicken and daiya cheese and only ...,"['ordered', 'vegan', 'chicken', 'daiya', 'chee...",36,False,"[order vegan chicken, daiya cheese, only wait,..."
348846,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,1,1,0,This was the worst pizza I've ever had ever ea...,2019-07-08 19:27:04,this be the bad pizza ever have ever eatenwe o...,"['worst', 'pizza', 'ive', 'ever', 'ever', 'eat...",51,False,[this be the bad pizza ever have ever eatenwe ...
348847,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,3,0,1,One of the best pizzas in SB....seriously!!!\n...,2014-07-07 18:40:39,one of the good pizza in sbseriouslythis be a ...,"['one', 'best', 'pizza', 'sbseriously', 'regul...",116,False,[one of the good pizza in sbseriouslythis be a...


In [12]:
def filter_phrases_with_keywords(phrases, aspect_keywords):
    filtered_phrases = []
    deleted_phrases = []
    for phrase in phrases:
        aspect_found = False
        for aspect, keywords in aspect_keywords.items():
            if any(keyword.lower() in phrase.lower() for keyword in keywords):
                filtered_phrases.append(phrase)
                aspect_found = True
                break
        if not aspect_found:
            deleted_phrases.append(phrase)
    return filtered_phrases, deleted_phrases


In [13]:
with open('categories.txt', 'r') as file:
    contents = file.read()
aspect_keywords_dict = eval(contents)

In [14]:
aspect_keywords =aspect_keywords_dict

In [16]:
from collections import defaultdict

def create_aspect_columns(df, aspect_keywords):
    aspect_dict = defaultdict(list)
    
    for aspect, keywords in aspect_keywords.items():
        for keyword in keywords:
            aspect_dict[keyword.lower()].append(aspect)

    for aspect in aspect_keywords.keys():
        df[aspect] = ''
    
    for i in range(len(df)):
        for phrase in df.loc[i, 'phrases']:
            for word in phrase.split():
                word = word.lower()
                if word in aspect_dict:
                    for aspect in aspect_dict[word]:
                        df.at[i, aspect] = phrase
                    break
                    
    return df

In [17]:
df = df.reset_index(drop=True)

df = create_aspect_columns(df, aspect_keywords)
df

,business_id,location,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,tokens,num_tokens,has_emoji,phrases,food,service,price,atmosphere,alternative diet,locationCategory
0,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,"['great', 'addition', 'funk', 'zone', 'grab', ...",24,False,"[, a great addition to the funk zone grab a bi...",a great addition to the funk zone grab a bite ...,,,,,
1,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,"['possibly', 'best', 'breakfast', 'sandwich', ...",9,False,[possibly the good breakfast sandwich ever on ...,possibly the good breakfast sandwich ever on a...,,,,,
2,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,"['pricey', 'ham', 'cheese', 'croissant', '550'...",46,False,"[pricey a ham, cheese croissant be, with anywh...",pesto sammy be also great go back,,pricey a ham,,,
3,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,"['kidding', 'without', 'doubt', 'best', 'food'...",30,False,[be you kid I this be without a doubt some of ...,kind coffee look great,,I try it I will be price be good too,very chic people be friendly,,
4,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,34.414445,-119.690672,4.0,389,...,"['super', 'slow', 'service', 'salad', 'good', ...",9,False,"[super slow service be not very good, really o...",,super slow service be not very good,really over price be return,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239060,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,"['one', 'new', 'favorite', 'pizza', 'joint', '...",69,False,[this be one of my new favorite pizza joint he...,the crust come out the same on their whole piz...,both be great I have only order their pizza by...,,,,the first time about a month ago I live in lon...
239061,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,"['ordered', 'vegan', 'chicken', 'daiya', 'chee...",36,False,"[order vegan chicken, daiya cheese, only wait,...",min to get this freshly bake pizza the crust b...,satisfying service be excellent the waiter let...,,friend gather hope to come back again soon,,
239062,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,"['worst', 'pizza', 'ive', 'ever', 'ever', 'eat...",51,False,[this be the bad pizza ever have ever eatenwe ...,a greasy mess the crust be thin,,,my family with a hot car full of sick people,,
239063,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105.0,34.440689,-119.739681,4.0,299,...,"['one', 'best', 'pizza', 'sbseriously', 'regul...",116,False,[one of the good pizza in sbseriouslythis be a...,the buffalo chicken each be pretty unique,you can manage to order during happy hour thin...,entertainment purpose price cheap,we can watch sport,plus it just plain taste good they also have g...,


### Textblob

In [18]:
from textblob import TextBlob

def get_sentiment_score(text):
    return TextBlob(text).sentiment.polarity


In [65]:
#from nltk.sentiment.vader import SentimentIntensityAnalyzer

#def get_sentiment_score(text):
#    sia = SentimentIntensityAnalyzer()
#    sentiment_dict = sia.polarity_scores(text)
#    return sentiment_dict['compound']

In [19]:
df['food_sentiment'] = df['food'].apply(get_sentiment_score)
df['service_sentiment'] = df['service'].apply(get_sentiment_score)
df['atmosphere_sentiment'] = df['atmosphere'].apply(get_sentiment_score)
df['price_sentiment'] = df['price'].apply(get_sentiment_score)
df['alternative_diet_sentiment'] = df['alternative diet'].apply(get_sentiment_score)
df['locationCategory_sentiment'] = df['locationCategory'].apply(get_sentiment_score)


In [67]:
# assume `df` is the pandas dataframe you want to output
#file_path = "/Users/datascience/Desktop/Cap/pre_pivot.csv"
df.to_csv('pre_pivot.csv', index=False)



Total

In [68]:
# Group by location and calculate the average sentiment score for each aspect keyword
grouped_df = df.groupby('location').mean().reset_index()

# Select only the location and aspect sentiment columns
location_sentiment_df = grouped_df[['location'] + [col for col in grouped_df.columns if '_sentiment' in col]]

# Print the result
location_sentiment_df

,location,food_sentiment,service_sentiment,atmosphere_sentiment,price_sentiment,alternative_diet_sentiment,locationCategory_sentiment
0,101 Deli,0.223270,0.085863,0.006771,0.088609,0.000000,-0.006622
1,1114 Sports Bar & Games,0.210958,0.138065,0.134607,0.046059,0.004167,0.039415
2,212° Hotpot,0.159699,0.062246,0.071438,0.066590,0.005068,0.005574
3,4 Eggs & Pizza,0.274789,0.031636,0.077675,0.004672,0.013707,0.000000
4,6831 AA Hollister Ave,0.085784,0.089846,0.047500,-0.004739,0.000000,-0.007598
...,...,...,...,...,...,...,...
1388,duo events,0.293056,0.196250,0.001488,0.013333,0.000000,0.000000
1389,goa taco,0.193142,0.044686,0.031915,0.040338,0.029478,0.010411
1390,iGrill Korean BBQ,0.132126,0.075604,0.058161,0.037082,0.005405,0.008706
1391,il Fustino,0.147279,0.211938,0.049594,0.100167,0.017857,-0.002269


In [69]:
# output DF to a csv
# assume `df` is the pandas dataframe you want to output
#file_path = "/Users/datascience/Desktop/Cap/post_pivot.csv"
location_sentiment_df.to_csv('post_pivot.csv', index=False)

